# Ниязов Алдияр М19-04
Задание 2. Факторизационная машина. Netflix dataset

In [4]:
from scipy import sparse
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import time
import os

## Подготовка данных

из описания датасета опредлим количество пользователей, фильмов и рейтингов

In [2]:
n_movies = 17770
n_users = 2649429
n_rate = 100480508

Определим горизонтальную размерность матрицы признаков после кодирования, как сумму количества пользователей и фильмов

In [3]:
n = n_movies + n_users

Ниже приведен код парсера, который итерируясь по всем файлам исходников соберет общую матрицу данных в виде: one-hot-codding фильма, one-hot-codding фильма и соответствующий рейтинг. Так как матрица будет сильно разреженной, то для экономии памяти будем использовать формат csr_sparse

In [ ]:
# определям директорию где находятся исходники
directory = os.fsencode('training_set')

# инициализируем нулями строку матрицы соответствующую одному наблюдению
features = sparse.csr_matrix((1, n + 1), dtype=np.int8)

files_end = 0
data = []
row = []
col = []
i = 0
# итерируемся по файлам
for file in os.listdir(directory):

    filename = os.fsdecode(file)

    with open(f'training_set/{filename}', 'r') as f:
        text = f.readlines()
    # определяем id фильма
    movie_id = int(text[0].split(':')[0])
    
    # итерируемся по строкам в файле
    for line in text[1:]:
        info = line.split(',')
        # определяем id пользователя и рейтинг
        user_id = int(info[0])
        rating = int(info[1])
        # собираем данные в соответствующие массивы
        data.append(1)
        data.append(1)
        data.append(rating)

        row.append(i)
        row.append(i)
        row.append(i)
        
        # расположение в строке единички для фильма
        col.append(movie_id - 1)
        # так как в строке код пользователя следует после кода фильма,
        # то id пользователя смещается на 17770
        col.append(user_id + n_movies - 1)
        # расположение в строке рейтинга
        col.append(n)

        i += 1

    files_end += 1
    if files_end % 1000 == 0:
        # из собранных массивов определяем промежуточную спарс-матрицу
        temp_matrix = sparse.csr_matrix((data, (row, col)), dtype=np.int8, shape=(i, n + 1))
        # стакаем предыдущие значения итоговой матрицы с промежуточной
        features = sparse.vstack([features, temp_matrix], dtype=np.int8)
        print(files_end)
        # обнуляем массивы
        data = []
        row = []
        col = []
        i = 0
# стакаем остатки значений
temp_matrix = sparse.csr_matrix((data, (row, col)), dtype=np.int8, shape=(i, n + 1))
features = sparse.vstack([features, temp_matrix], dtype=np.int8)

# сохраняем полученную матрицу данных на диск
sparse.save_npz('sparse_matrix.npz', features)

In [3]:
dataset = sparse.load_npz('sparse_matrix.npz')

перемешиваем данные

In [5]:
dataset = shuffle(dataset)

In [6]:
sparse.save_npz('shuffle_dataset.npz', dataset)

разделяем матрицу признаков и целевые значения

In [ ]:
all_cols = np.arange(dataset.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, all_cols[:n])))[0]
target = dataset[:, cols_to_keep]
features = dataset[:, all_cols[:n]]

sparse.save_npz('features.npz', features)
sparse.save_npz('target.npz', target)

In [5]:
features

<100480508x2667199 sparse matrix of type '<class 'numpy.int8'>'
	with 200961014 stored elements in Compressed Sparse Row format>

Для проведения кросс-валидации разбиваем признаки и целевые значения на 5 частей примерно по 20млн. в каждой

In [4]:
features = sparse.load_npz('features.npz')

In [5]:
sparse.save_npz('features_1.npz', features[:20000000])
sparse.save_npz('features_2.npz', features[20000000:40000000])
sparse.save_npz('features_3.npz', features[40000000:60000000])
sparse.save_npz('features_4.npz', features[60000000:80000000])
sparse.save_npz('features_5.npz', features[80000000:])

In [6]:
target = sparse.load_npz('target.npz')

In [7]:
sparse.save_npz('target_1.npz', target[:20000000])
sparse.save_npz('target_2.npz', target[20000000:40000000])
sparse.save_npz('target_3.npz', target[40000000:60000000])
sparse.save_npz('target_4.npz', target[60000000:80000000])
sparse.save_npz('target_5.npz', target[80000000:])

## обучение модели и расчет метрик

обучать факторизационную машину будем мини-батч градиентным спуском

In [4]:
def grad_descent_1(X, y, n, lr, epoch_num):
    # инициализируем матрицу факторов V, вектор весов признаков w и w0
    V = np.random.random(size=(n, 3))
    w0 = 0
    w = np.random.random(size=(n, 1))
    
    # определяем эпохи
    for epoch in range(epoch_num):
        start = time.time()
        # перемешиваем данные
        X, y = shuffle(X, y)
        # определяем размер батча
        batch_size = 90000
        # проходим батчами по датасету
        for batch_idx in range(int(X.shape[0] / batch_size + 1)):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size
            batch = X[start_idx:end_idx]
            y_batch = y[start_idx:end_idx]
            
            # рассчитываем постоянный компонент для экономии времени в дальнейшем
            batch_dot_v = batch.dot(V)
            # делаем прогноз
            y_pred = w0 + batch.dot(w) + (0.5 * np.sum((batch_dot_v ** 2) - batch.dot(V ** 2), axis=1)).reshape(batch.shape[0], 1)
            # рассчитываем градиент функции потерь (MSE)
            dL_dy = 2 * (y_pred.reshape(y_batch.shape[0], 1) - y_batch.reshape(y_batch.shape[0], 1))
            # рассчитываем градиенты по параметрам
            dL_dw0 = dL_dy.mean()
            dL_dw = batch.T.dot(dL_dy)
            dL_dv = (batch.T.dot(batch_dot_v) - np.sum((batch.multiply(batch)), axis=0) * V) * dL_dw0 / batch.shape[0]
            
            # обновляем параметры
            w0 -= lr * dL_dw0
            w -= lr * dL_dw
            V -= lr * dL_dv

        stop = time.time()
        print(f'epoch #{epoch} time: {stop - start} seconds')

    return w0, w, V

определяем функции для расчета прогноза и метрики

In [5]:
def get_prediction(X, w0, w, V):
    return w0 + X.dot(w) + (0.5 * np.sum((X.dot(V) ** 2) - X.dot(V**2), axis=1)).reshape(X.shape[0],1)

In [6]:
def get_rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

так как моя реализация и ресурсы моей машины не позволяют реализовать полную кросс-валидацию в одном цикле, то будем производить ее по частям

In [7]:
feature_fold = [
    'features_1.npz',
    'features_2.npz',
    'features_3.npz',
    'features_4.npz',
    'features_5.npz'
]
target_fold = [
    'target_1.npz',
    'target_2.npz',
    'target_3.npz',
    'target_4.npz',
    'target_5.npz'
]

In [8]:
def cross_validation(feature_fold, target_fold, idx):
    metrics = []
    weights = []
    
    print(f"\n validation #{idx}\n")
    data_train = []
    target_train = []
    # заружаем с диска часть данных для теста
    data_test = sparse.load_npz(feature_fold.pop(idx))
    target_test = sparse.load_npz(target_fold.pop(idx)).todense()
    # объединяем оставшиеся части для обучения
    for i in feature_fold:
        data_train.append(sparse.load_npz(i))
    for j in target_fold:
        target_train.append(sparse.load_npz(j))

    data_train = sparse.vstack(data_train)
    target_train = sparse.vstack(target_train).todense()

    # обучение факторизационной машины
    w0, w, V = grad_descent_1(data_train, target_train, n, 0.0001, 5)
    # расчет метрик
    y_pred_train = get_prediction(data_train, w0, w, V)
    y_pred_test = get_prediction(data_test, w0, w, V)
    RMSE_train = get_rmse(target_train, y_pred_train)
    RMSE_test = get_rmse(target_test, y_pred_test)
    
    # объединение результатов
    metrics.append([RMSE_train, RMSE_test])
    weights.append([w0, w, V])
    print(f'RMSE train: {RMSE_train}')
    print(f'RMSE test: {RMSE_test}')

    return metrics, weights

In [9]:
metrics_0, weights_0 = cross_validation(feature_fold, target_fold, idx=0)


 validation #0

epoch #0 time: 825.5817205905914 seconds
epoch #1 time: 1941.5300951004028 seconds
epoch #2 time: 1867.6005456447601 seconds
epoch #3 time: 1862.4598729610443 seconds
epoch #4 time: 1818.3506247997284 seconds
RMSE train: 1.0340010768324148
RMSE test: 1.0354794566890753


In [9]:
metrics_1, weights_1 = cross_validation(feature_fold, target_fold, idx=1)


 validation #1

epoch #0 time: 778.4358777999878 seconds
epoch #1 time: 1720.4833624362946 seconds
epoch #2 time: 1798.7089726924896 seconds
epoch #3 time: 2115.09365773201 seconds
epoch #4 time: 2141.9165403842926 seconds
RMSE train: 1.0340312433924348
RMSE test: 1.0349081860912048


In [9]:
metrics_2, weights_2 = cross_validation(feature_fold, target_fold, idx=2)


 validation #2

epoch #0 time: 882.2516176700592 seconds
epoch #1 time: 2023.3425316810608 seconds
epoch #2 time: 1870.3911740779877 seconds
epoch #3 time: 2066.297188282013 seconds
epoch #4 time: 1725.9794397354126 seconds
RMSE train: 1.033638900611023
RMSE test: 1.034918912076469


In [9]:
metrics_3, weights_3 = cross_validation(feature_fold, target_fold, idx=3)


 validation #3

epoch #0 time: 900.4800293445587 seconds
epoch #1 time: 1717.0699076652527 seconds
epoch #2 time: 1716.3005232810974 seconds
epoch #3 time: 2793.074732065201 seconds
epoch #4 time: 2427.108398914337 seconds
RMSE train: 1.0335742903131206
RMSE test: 1.0343878814603689


In [30]:
metrics_4, weights_4 = cross_validation(feature_fold, target_fold, idx=4)


 validation #4

epoch #0 time: 753.9998817443848 seconds
epoch #1 time: 1195.5279302597046 seconds
epoch #2 time: 2077.0152752399445 seconds
epoch #3 time: 2307.4723348617554 seconds
epoch #4 time: 2106.04757976532 seconds
RMSE train: 1.033992909612587
RMSE test: 1.034857833837001


## представление результатов

количество эпох: 5
размер батча: 90 000
скорость обучения: 0.0001
количество фолдов: 5
количество признаков: 2 667 199
количество факторов: 3

In [6]:
data_metrics = {
        'fold 1': metrics_0, 
        'fold 2': metrics_1, 
        'fold 3': metrics_2, 
        'fold 4': metrics_3, 
        'fold 5': metrics_4,
        'mean': mean,
        'std': std
       }

df_metrics = pd.DataFrame(data_metrics, index=['RMSE train', 'RMSE test'])

метрики:

In [7]:
df_metrics

,fold 1,fold 2,fold 3,fold 4,fold 5,mean,std
RMSE train,1.0340,1.0340,1.0336,1.0335,1.0339,1.03380,0.00021
RMSE test,1.0354,1.0349,1.0349,1.0343,1.0348,1.03486,0.00035
